In [3]:
import spacy
import networkx as nx
import matplotlib.pyplot as plt

# 1. Análisis de Spacy

In [5]:
text ='London is the capital and largest city of England and the United Kingdom. Standing on the River Thames in the south-east of England, at the head of its 50-mile (80 km) estuary leading to the North Sea, London has been a major settlement for two millennia. Londinium was founded by the Romans. The City of London, London’s ancient core − an area of just 1.12 square miles (2.9 km2) and colloquially known as the Square Mile − retains boundaries that follow closely its medieval limits.The City of Westminster is also an Inner London borough holding city status. Greater London is governed by the Mayor of London and the London Assembly.London is located in the southeast of England.Westminster is located in London.London is the biggest city in Britain. London has a population of 7,172,036.'

In [6]:
spc = spacy.load('en_core_web_sm')

In [7]:
doc = spc(text)

In [8]:
print('Noun phrases:',[chunk.text for chunk in doc.noun_chunks])

Noun phrases: ['London', 'the capital', 'largest city', 'England', 'the United Kingdom', 'the River Thames', 'the south-east', 'England', 'the head', 'the North Sea', 'London', 'a major settlement', 'two millennia', 'Londinium', 'the Romans', 'The City', 'London', 'London', 'ancient core', 'an area', 'just 1.12 square miles', '2.9 km2', 'the Square Mile −', 'boundaries', 'its medieval limits', 'The City', 'Westminster', 'an Inner London borough holding city status', 'Greater London', 'the Mayor', 'London', 'the London Assembly', 'London', 'the southeast', 'England', 'Westminster', 'London', 'London', 'the biggest city', 'Britain', 'London', 'a population']


In [9]:
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Verbs: ['stand', 'lead', 'found', 'know', 'retain', 'follow', 'hold', 'govern', 'locate', 'locate']


In [10]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

London GPE
England GPE
the United Kingdom GPE
the River Thames ORG
England GPE
50-mile MONEY
80 km QUANTITY
the North Sea LOC
London GPE
two millennia DATE
Romans NORP
The City of London GPE
London GPE
− PERSON
just 1.12 square miles QUANTITY
2.9 km2 QUANTITY
Square Mile − LOC
The City of Westminster GPE
Inner NORP
London GPE
Greater London PERSON
the Mayor of London ORG
the London Assembly ORG
London GPE
England GPE
London GPE
London GPE
Britain GPE
London GPE
7,172,036 CARDINAL


In [ ]:
spacy.displacy.serve(doc, style='dep')

/home/jennifer/insaite/virtual_envs/AI_Recruitment_venv/lib/python3.8/site-packages/spacy/displacy/__init__.py:94: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



# 2. Funciones

In [4]:
def getSentences(text):
    nlp = spacy.load('en_core_web_sm')
    nlp.add_pipe(nlp.create_pipe('sentencizer'))
    document = nlp(text)
    return [sent.string.strip() for sent in document.sents]
def printToken(token):
    print(token.text, "->", token.dep_)
    
def appendChunk(original, chunk):
    return original + ' ' + chunk